# (1) Clean The CAD Data

In [14]:
#imports
import pandas as pd
from pathlib import Path

In [15]:
# Bring in the CAD Data and select relevant columns
CAD_data = pd.read_csv("data/call_data_from_CAD.csv")
CAD_data = CAD_data[["Call_Created_Time", "Call_Zipcode", "InitialIncidentTypeDescription", "Disposition",
                     "PrimaryUnitCallSign", "RespondingUnitCallSign"]].copy()

# Convert call times to Date Time objects
CAD_data["Call_Created_Time"] = pd.to_datetime(CAD_data['Call_Created_Time'])
CAD_data

,Call_Created_Time,Call_Zipcode,InitialIncidentTypeDescription,Disposition,PrimaryUnitCallSign,RespondingUnitCallSign
0,2016-01-01 00:00:04,97402.0,ASSAULT,ADVISED,5E57,5E57
1,2016-01-01 00:00:04,97402.0,ASSAULT,ADVISED,5E57,4X40
2,2016-01-01 00:00:04,97402.0,ASSAULT,ADVISED,5E57,4E53
3,2016-01-01 00:00:24,97401.0,TRAFFIC STOP,ADVISED,5T81,5T81
4,2016-01-01 00:02:45,97401.0,CHECK WELFARE,ASSISTED,3X90,3F61
...,...,...,...,...,...,...
1616824,2023-12-31 23:23:05,97402.0,CRIMINAL TRESPASS,DISREGARD,NaN,NaN
1616825,2023-12-31 23:26:00,97401.0,AUDIBLE ALARM,INFORMATION ONLY,NaN,NaN
1616826,2023-12-31 23:32:06,97408.0,SUSPICIOUS CONDITIONS,QUALITY OF LIFE - NO DISPATCH,NaN,NaN
1616827,2023-12-31 23:33:30,97401.0,SHOTS FIRED,REFERRED TO OTHER AGENCY,NaN,NaN


### Standardize Cahoots identifiers 

In [16]:
# Standardize Cahoots identifiers 
cahoots_identifiers = r"1J77\s*|3J79\s*|3J78\s*|3J77\s*|4J79\s*|3J81\s*|3J76\s*|2J28\s*|2J29\s*|CAHOOT\s*|CAHOT\s*|CAHO\s*"

CAD_data["PrimaryUnitCallSign"] = CAD_data["PrimaryUnitCallSign"].replace(cahoots_identifiers, 'CAHOOT', regex=True)
CAD_data["RespondingUnitCallSign"] = CAD_data["RespondingUnitCallSign"].replace(cahoots_identifiers, 'CAHOOT', regex=True)

# Create an identifier for Cahoots involvement 
CAD_data['Handled_by_Cahoots'] = ((CAD_data['PrimaryUnitCallSign'] == 'CAHOOT') | (CAD_data['RespondingUnitCallSign'] == 'CAHOOT')).astype(int)

### Remove Disregards, duplicates, referrals and cancellations

In [17]:
# Disregards
CAD_data = CAD_data[CAD_data['Disposition'] != 'DISREGARD']
CAD_data = CAD_data[CAD_data['Disposition'] != 'DISREGARDED BY DISPATCH']
CAD_data = CAD_data[CAD_data['Disposition'] != 'DISREGARDED BY PATROL SUPERVISOR']

# Duplicate or no Dispatch
CAD_data = CAD_data[CAD_data['Disposition'] != 'ACCIDENTALLY CHOSE NEW EVENT']
CAD_data = CAD_data[CAD_data['Disposition'] != 'MOTOR VEHICLE ACCIDENT - NO DISPATCH']
CAD_data = CAD_data[CAD_data['Disposition'] != 'QUALITY OF LIFE - NO DISPATCH']
CAD_data = CAD_data[CAD_data['Disposition'] != 'UNABLE TO DISPATCH']
CAD_data = CAD_data[CAD_data['Disposition'] != 'WILL CALL BACK']
CAD_data = CAD_data[~(CAD_data["PrimaryUnitCallSign"].isna()) & ~(CAD_data["RespondingUnitCallSign"].isna())]

# Cancellations
CAD_data = CAD_data[CAD_data['Disposition'] != 'NO ACTION TAKEN']
CAD_data = CAD_data[CAD_data['Disposition'] != 'CANCEL WHILE ENROUTE']
CAD_data = CAD_data[CAD_data['Disposition'] != 'RESOLVED']
CAD_data = CAD_data[CAD_data['Disposition'] != 'CANCELED REPORT NUMBER']
CAD_data = CAD_data[CAD_data['Disposition'] != 'CANCEL FIRE UNIT FROM CALL']

# Referrals and relays
CAD_data = CAD_data[CAD_data['Disposition'] != 'REFERRED TO OTHER AGENCY']
CAD_data = CAD_data[CAD_data['Disposition'] != 'RELAYED TO UNIVERSITY OF OREGON POLICE']
CAD_data = CAD_data[CAD_data['Disposition'] != 'RELAYED TO OREGON STATE POLICE']
CAD_data = CAD_data[CAD_data['Disposition'] != 'RELAYED TO LANE COUNTY SHERIFFS OFFICE']
CAD_data = CAD_data[CAD_data['Disposition'] != 'RELAYED TO PARKING CONTROL']

CAD_data = CAD_data[CAD_data['Call_Created_Time'].dt.year != 2023]

CAD_data

,Call_Created_Time,Call_Zipcode,InitialIncidentTypeDescription,Disposition,PrimaryUnitCallSign,RespondingUnitCallSign,Handled_by_Cahoots
0,2016-01-01 00:00:04,97402.0,ASSAULT,ADVISED,5E57,5E57,0
1,2016-01-01 00:00:04,97402.0,ASSAULT,ADVISED,5E57,4X40,0
2,2016-01-01 00:00:04,97402.0,ASSAULT,ADVISED,5E57,4E53,0
3,2016-01-01 00:00:24,97401.0,TRAFFIC STOP,ADVISED,5T81,5T81,0
4,2016-01-01 00:02:45,97401.0,CHECK WELFARE,ASSISTED,3X90,3F61,0
...,...,...,...,...,...,...,...
1616471,2022-12-31 23:54:01,NaN,SHOTS FIRED,PATROL CHECK,4E36,6E57,0
1616472,2022-12-31 23:54:01,NaN,SHOTS FIRED,PATROL CHECK,4E36,6E27,0
1616473,2022-12-31 23:54:01,NaN,SHOTS FIRED,PATROL CHECK,4E36,4E36,0
1616474,2022-12-31 23:54:01,NaN,SHOTS FIRED,PATROL CHECK,4E36,CMD10,0


In [18]:
CAD_data = CAD_data.sort_values(by="Call_Created_Time")
output_dir = Path("data/cleaned_data")
output_dir.mkdir(parents=True, exist_ok=True)

# Save the cleaned DataFrame 
output_path = output_dir / "cleaned_CAD_data.csv"
CAD_data.to_csv(output_path, index=False)

print(f"Cleaned data saved to {output_path}")

Cleaned data saved to data\cleaned_data\cleaned_CAD_data.csv


# (2) Clean Cahoots Data

In [19]:
# Load Data
cahoots_dataset = pd.read_excel("data/call_data_from_CAHOOTS_2021_2022.xlsx")

In [20]:
# Drop NA
cahoots_dataset = cahoots_dataset.dropna(subset=["Reason for Dispatch"])

# Combine Date and TimeOfCall into a single DateTime column
cahoots_dataset["Date"] = cahoots_dataset["Date"].astype(str)
cahoots_dataset["TimeOfCall"] = cahoots_dataset["TimeOfCall"].astype(str)
cahoots_dataset["DateTime"] = pd.to_datetime(cahoots_dataset["Date"] + " " + cahoots_dataset["TimeOfCall"], errors='coerce')

# Only select data from Eugene
cahoots_dataset = cahoots_dataset[cahoots_dataset["City"] == "Eugene"]

# Drop unneeded columns
cahoots_dataset = cahoots_dataset[["DateTime", "Reason for Dispatch"]]

In [21]:
cahoots_dataset

,DateTime,Reason for Dispatch
0,2021-01-04 20:10:31,Public Assist
5,2021-01-14 17:10:00,Check Welfare
6,2021-01-14 17:34:00,Check Welfare
7,2021-01-14 18:01:00,Public Assist
8,2021-01-14 19:20:00,Check Welfare
...,...,...
46887,2022-12-31 20:45:01,Public Assist
46888,2022-12-31 21:09:05,Check Welfare
46889,2022-12-31 21:26:08,Public Assist
46890,2022-12-31 21:37:01,Public Assist


In [22]:
# Save File
output_path = output_dir / "cleaned_cahoots_data.csv"
cahoots_dataset.to_csv(output_path, index=False)

print(f"Cleaned data saved to {output_path}")

Cleaned data saved to data\cleaned_data\cleaned_cahoots_data.csv


# Create CAD Diversions Dataset

In [23]:
CAD_data = pd.read_csv("data\cleaned_data\cleaned_CAD_data.csv")
CAD_data

,Call_Created_Time,Call_Zipcode,InitialIncidentTypeDescription,Disposition,PrimaryUnitCallSign,RespondingUnitCallSign,Handled_by_Cahoots
0,2016-01-01 00:00:04,97402.0,ASSAULT,ADVISED,5E57,5E57,0
1,2016-01-01 00:00:04,97402.0,ASSAULT,ADVISED,5E57,4X40,0
2,2016-01-01 00:00:04,97402.0,ASSAULT,ADVISED,5E57,4E53,0
3,2016-01-01 00:00:24,97401.0,TRAFFIC STOP,ADVISED,5T81,5T81,0
4,2016-01-01 00:02:45,97401.0,CHECK WELFARE,ASSISTED,3X90,3F61,0
...,...,...,...,...,...,...,...
1104434,2022-12-31 23:54:01,NaN,SHOTS FIRED,PATROL CHECK,4E36,CMD10,0
1104435,2022-12-31 23:54:01,NaN,SHOTS FIRED,PATROL CHECK,4E36,4K98,0
1104436,2022-12-31 23:54:01,NaN,SHOTS FIRED,PATROL CHECK,4E36,5E59,0
1104437,2022-12-31 23:54:01,NaN,SHOTS FIRED,PATROL CHECK,4E36,3E27,0


### Remove all call types with 0 overlap between Police and Cahoots

In [24]:
cahoots_related = CAD_data[CAD_data['Handled_by_Cahoots'] == 1]
police_handled = CAD_data[CAD_data['Handled_by_Cahoots'] == 0]

# Unique incident types
cahoots_types = cahoots_related['InitialIncidentTypeDescription'].unique()
police_types = police_handled['InitialIncidentTypeDescription'].unique()

# Incident types never responded to by Cahoots
never_cahoots_type = list(set(police_types) - set(cahoots_types))

# Incident types never responded to by police
never_police_type = list(set(cahoots_types) - set(police_types))

# Unique dispositions
cahoots_disp = cahoots_related['Disposition'].unique()
police_disp = police_handled['Disposition'].unique()

# Dispositions never responded to by Cahoots
never_cahoots_disp = list(set(police_disp) - set(cahoots_disp))

# Dispositions never responded to by police
never_police_disp = list(set(cahoots_disp) - set(police_disp))

# Filter out incidents and dispositions never responded to by Cahoots or police
CAD_data_diversions = CAD_data[
    ~CAD_data['InitialIncidentTypeDescription'].isin(never_cahoots_type) &
    ~CAD_data['InitialIncidentTypeDescription'].isin(never_police_type)
]
CAD_data_diversions = CAD_data_diversions[
    ~CAD_data_diversions['Disposition'].isin(never_cahoots_disp) &
    ~CAD_data_diversions['Disposition'].isin(never_police_disp)
]
CAD_data_diversions

,Call_Created_Time,Call_Zipcode,InitialIncidentTypeDescription,Disposition,PrimaryUnitCallSign,RespondingUnitCallSign,Handled_by_Cahoots
0,2016-01-01 00:00:04,97402.0,ASSAULT,ADVISED,5E57,5E57,0
1,2016-01-01 00:00:04,97402.0,ASSAULT,ADVISED,5E57,4X40,0
2,2016-01-01 00:00:04,97402.0,ASSAULT,ADVISED,5E57,4E53,0
3,2016-01-01 00:00:24,97401.0,TRAFFIC STOP,ADVISED,5T81,5T81,0
4,2016-01-01 00:02:45,97401.0,CHECK WELFARE,ASSISTED,3X90,3F61,0
...,...,...,...,...,...,...,...
1104434,2022-12-31 23:54:01,NaN,SHOTS FIRED,PATROL CHECK,4E36,CMD10,0
1104435,2022-12-31 23:54:01,NaN,SHOTS FIRED,PATROL CHECK,4E36,4K98,0
1104436,2022-12-31 23:54:01,NaN,SHOTS FIRED,PATROL CHECK,4E36,5E59,0
1104437,2022-12-31 23:54:01,NaN,SHOTS FIRED,PATROL CHECK,4E36,3E27,0


### Remove call types below below composite score threshold

In [25]:
from scipy.stats import zscore
import numpy as np

# Calculate total calls, Cahoots calls, and police calls for each type in CAD_data_diversions
total_calls = CAD_data_diversions['InitialIncidentTypeDescription'].value_counts()
cahoots_calls = CAD_data_diversions[CAD_data_diversions['Handled_by_Cahoots'] == 1]['InitialIncidentTypeDescription'].value_counts()
police_calls = CAD_data_diversions[CAD_data_diversions['Handled_by_Cahoots'] == 0]['InitialIncidentTypeDescription'].value_counts()

# Calculate proportions of Cahoots and police involvement in CAD_data_diversions
cahoots_proportion = cahoots_calls / total_calls
police_proportion = police_calls / total_calls

# Apply optional scaling
scaled_cahoots_calls = np.sqrt(cahoots_calls)
scaled_police_calls = np.sqrt(police_calls)

# Calculate harmonic mean of the proportions
harmonic_mean_proportion = 2 * (cahoots_proportion * police_proportion) / (cahoots_proportion + police_proportion + 1e-10)  # Adding a small value to avoid division by zero

# Composite score based on harmonic mean and call counts
composite_scores = harmonic_mean_proportion * ((scaled_cahoots_calls + scaled_police_calls))

# Apply z-score normalization to the composite scores
normalized_composite_scores = zscore(composite_scores)

# normalized composite scorethreshold
substantial_incident_types = composite_scores[normalized_composite_scores > 0.5].index

# Filter data
filtered_CAD_data_diversions = CAD_data_diversions[CAD_data_diversions['InitialIncidentTypeDescription'].isin(substantial_incident_types)]

filtered_CAD_data_diversions

,Call_Created_Time,Call_Zipcode,InitialIncidentTypeDescription,Disposition,PrimaryUnitCallSign,RespondingUnitCallSign,Handled_by_Cahoots
4,2016-01-01 00:02:45,97401.0,CHECK WELFARE,ASSISTED,3X90,3F61,0
5,2016-01-01 00:02:45,97401.0,CHECK WELFARE,ASSISTED,3X90,4F72,0
6,2016-01-01 00:02:45,97401.0,CHECK WELFARE,ASSISTED,3X90,4F71,0
7,2016-01-01 00:02:45,97401.0,CHECK WELFARE,ASSISTED,3X90,3X90,0
14,2016-01-01 00:07:19,97401.0,DISORDERLY SUBJECT,ASSISTED,6E26,4E23,0
...,...,...,...,...,...,...,...
1104396,2022-12-31 21:53:47,NaN,CHECK WELFARE,ARREST,4F72,4E36,0
1104397,2022-12-31 21:53:47,NaN,CHECK WELFARE,ARREST,4F72,4E54,0
1104418,2022-12-31 23:12:35,NaN,ASSIST FIRE DEPARTMENT,ASSISTED,4E11,6E27,0
1104419,2022-12-31 23:12:35,NaN,ASSIST FIRE DEPARTMENT,ASSISTED,4E11,4E36,0


In [26]:
# Save File 
output_path = output_dir / "cleaned_CAD_diversions.csv"
filtered_CAD_data_diversions.to_csv(output_path, index=False)

print(f"Cleaned data saved to {output_path}")

Cleaned data saved to data\cleaned_data\cleaned_CAD_diversions.csv
